In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as mtick
import statsmodels.api as sm
from scipy.interpolate import interp1d
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
%matplotlib inline

# USER INPUTS

In [ ]:
# Majority Level
# The majority level sets the percentage of the census tract population that must self-identify as the same 
# race/ethnicity in order for the census tract to be classified as that race/ethnicity. We had defined majority 
# as 50% and strong-majority as 75%.
majority_level = 50

# Variable of Interest
# For the LOWESS analysis, we explore racial disparity in rooftop PV deployment as a function of median household 
# income and the percentage of households occupied by renters. If you would like to generate plots with the median 
# household income along the x-axis (similar to Figures 2, 5, S1, and S5), use 'median_income' as your variable of 
# interest. If you would like to generate plots with the percentage of households occupied by renters (similar to 
# Figure 3), use 'rental_ratio' as your variable of interest. If you would like to generate plots with the median
# surplus household income along the x-axis (similar to Figure S2), use 'surplus_income' as your variable of interest.
variable_of_interest = 'median_income'

# Confidence Interval
# We used a 90% confidence interval (CI) in the published analysis but one may choose to explore a different CI.
CI = 90

# Exclude Census Tracts Without Rooftop PV Installations
# To explore the potential issue of 'seeding', we excluded census tracts without rooftop PV installations to create
# Figures 5 and S5. If you would like to reproduce these figures and exclude census tracts without rooftop PV use 
# True; otherwise, use False. Using False will include all census tracts in the analysis and allow you to reproduce 
# Figures 2, 3, and S1.
exclude_census_tracts_without_rooftop_PV = False

# Smoothing Parameter, f
# The subsets of data used for each weighted least squares fit in LOWESS are determined by a nearest neighbors 
# algorithm. A smoothing parameter, f, determines the fraction of the data that is used to fit each local regression.
# Large values of f produce the smoothest functions that wiggle the least in response to fluctuations in the data. The
# smaller f is, the closer the regression function will conform to the data. Using too small a value of f is not 
# desirable, however, since the regression function will eventually start to capture the random error in the data. 
# When the variable "smoothing_factor_default" is set to True the smoothing factor is set to 1 to be consistent with
# all reported figures in article. If you wish to set the smoothing parameter to another value, you may modify the 
# smoothing_parameter function in the code below. When this value is set to False, a range of smoothing parameters are
# considered and chosen to minimize the sum of the squared residuals.
smoothing_factor_default = True

# LOAD DATA

In [ ]:
# Demographic Data
# ACS 2015 Planning database available at https://www.census.gov/data/datasets/2015/adrm/research/2015-planning-database.html 
DemographicData = pd.read_csv('PDB_2015_Tract.csv', encoding = "ISO-8859-1")
DemographicData = DemographicData[['GIDTR', 'State_name', 
                                   'Renter_Occp_HU_ACS_09_13', 'Owner_Occp_HU_ACS_09_13', 
                                   'pct_NH_Asian_alone_ACS_09_13', 'pct_NH_Blk_alone_ACS_09_13',
                                   'pct_Hispanic_ACS_09_13', 'pct_NH_White_alone_ACS_09_13',
                                   'Med_HHD_Inc_ACS_09_13']]

In [ ]:
# Project Sunroof Data
# Data in this GitHub repository was updated in August 2021.
# More recently updated data can be found at https://www.google.com/get/sunroof/data-explorer/
SunroofData = pd.read_csv('project-sunroof-census_tract.csv')
SunroofData = SunroofData[['region_name', 'count_qualified', 'existing_installs_count', 'percent_covered', 'percent_qualified']]

In [ ]:
# Cost of Living Data
# Updated data can be found at https://livingwage.mit.edu/
# Data corresponds to the required annual income before taxes for a family of 2 adults and 2 children.
LivingWageData = pd.read_csv('cost_of_living_by_county.csv')
LivingWageData = LivingWageData[['county', 'required_income']]

# CLEAN DATA

In [ ]:
# Convert median household income from a string to a float and rename median_income
DemographicData['Med_HHD_Inc_ACS_09_13'] = DemographicData['Med_HHD_Inc_ACS_09_13'].replace({'\$': '', ',': ''}, regex=True).astype(float)
DemographicData = DemographicData.rename(columns={'Med_HHD_Inc_ACS_09_13': 'median_income'})

# Merge Google Project Sunroof data with Demographic data from ACS
data = SunroofData.merge(DemographicData,left_on='region_name',right_on='GIDTR')

# Remove census tracts where Google Project Sunroof analyzed < 95% of all buildings in the census tract.
data = data[data.percent_covered >= 95]

# Remove census tracts where there is no potential to install rooftop PV
data = data[data.count_qualified !=0]

# Remove census tracts with median household income below the 2013 poverty threshold for a 4-person household 
# https://www.census.gov/data/tables/time-series/demo/income-poverty/historical-poverty-thresholds.html
data = data[data.median_income >= 23834] 

data = data.reset_index(drop=True)

In [ ]:
# If the user input a request to remove census tracts without any rooftop solar from the analysis, remove these tracts
if exclude_census_tracts_without_rooftop_PV == True:
    data = data[data.existing_installs_count != 0]
    data = data.reset_index(drop=True)

In [ ]:
# If the user input would like to study surplus income, merge the Living Wage data.
# Note that the Living Wage data does not include US territories.
if variable_of_interest == 'surplus_income':
    for i in range(0,len(data)):
        data.at[i,'state_county'] = int(data.GIDTR[i].astype(str)[:-6])
    # Merge Living Wage data with the rest of the data
    data = data.merge(LivingWageData,left_on='state_county',right_on='county')
    data['surplus_income'] = data['median_income'] - data['required_income']
    data = data.reset_index(drop=True)

# CALCULATE STATE NORMALIZED SOLAR DEPLOYMENT

In [ ]:
# Equation 1 in the Nature Sustainability analysis
# Compute the percentage of buildings with existing rooftop PV relative to the total number of buildings with roofs 
# that qualify to support PV based on the algorithm and criteria to identify appropriate potential rooftop space for 
# PV deployment set forth by Google Project Sunroof. This is the solar deployment rate.
data['pct_current_installs_relative_to_total_installs']=data['existing_installs_count']/data['count_qualified']*100

In [ ]:
# Equation 2 in the Nature Sustainability analysis
# Normalize the solar deployment rate by the average solar deployment rate in each state.
# This mitigates the effects of variations across states, such as available solar resources, incentive programs and
# policies, and electricity prices.
df_state_mean = pd.DataFrame(columns=('State_name', 'state_mean_existing_installation_relative_to_potential'))

for state in set(data.State_name):
    mean_state = np.mean(data['pct_current_installs_relative_to_total_installs'][(data['State_name'] == state)])
    df_state_mean.loc[state] = [state, mean_state]

data = pd.merge(data, df_state_mean, on='State_name')
data['normalized_existing_installation']=data['pct_current_installs_relative_to_total_installs']/data['state_mean_existing_installation_relative_to_potential']

# Remove states or territories where Google Project Sunroof hasn't identified any rooftop solar installations
data = data[data.state_mean_existing_installation_relative_to_potential !=0]
data = data.reset_index(drop=True)

# GROUP CENSUS TRACTS BY THEIR MAJORITY RACE/ETHNICITY

In [ ]:
# Create dataframes for each race/ethnicity in the study
df_asian       = pd.DataFrame(columns=[variable_of_interest, 'normalized_existing_installation'])
df_black       = pd.DataFrame(columns=[variable_of_interest, 'normalized_existing_installation'])
df_hisp        = pd.DataFrame(columns=[variable_of_interest, 'normalized_existing_installation'])
df_white       = pd.DataFrame(columns=[variable_of_interest, 'normalized_existing_installation'])
df_no_majority = pd.DataFrame(columns=[variable_of_interest, 'normalized_existing_installation'])

# Calculate the fraction of renter occupied households
if variable_of_interest == 'rental_ratio':
    data['rental_ratio'] = 100*data.Renter_Occp_HU_ACS_09_13/(data.Renter_Occp_HU_ACS_09_13+data.Owner_Occp_HU_ACS_09_13)

# Filter data by majority level to populate each race/ethnicity dataframe
for i in range(0, len(data)):
    if data['pct_NH_Asian_alone_ACS_09_13'][i] >= majority_level:
        df_asian.loc[i] = [data[variable_of_interest][i], data['normalized_existing_installation'][i]]
    elif data['pct_NH_Blk_alone_ACS_09_13'][i] >= majority_level:
        df_black.loc[i] = [data[variable_of_interest][i], data['normalized_existing_installation'][i]]
    elif data['pct_Hispanic_ACS_09_13'][i] >= majority_level:
        df_hisp.loc[i] = [data[variable_of_interest][i], data['normalized_existing_installation'][i]]
    elif data['pct_NH_White_alone_ACS_09_13'][i] >= majority_level:
        df_white.loc[i] = [data[variable_of_interest][i], data['normalized_existing_installation'][i]]
    else:
        df_no_majority.loc[i] = [data[variable_of_interest][i], data['normalized_existing_installation'][i]]

# Reindex
df_asian = df_asian.reset_index(drop=True)
df_black = df_black.reset_index(drop=True)
df_hisp  = df_hisp.reset_index(drop=True)
df_white = df_white.reset_index(drop=True)
df_no_majority = df_no_majority.reset_index(drop=True)

# LOWESS ANALYSIS

## Functions

In [ ]:
# Perform LOWESS and determine the sum of the squared residuals
def lowess_sse(df, variable_of_interest, fraction):
    lowess = sm.nonparametric.lowess(df['normalized_existing_installation'], df[variable_of_interest], frac=fraction)
    lowess_x = list(zip(*lowess))[0]
    lowess_y = list(zip(*lowess))[1]
    squared_residuals = (df.normalized_existing_installation-lowess_y)**2
    sum_squared_residuals = sum(squared_residuals)
    return sum_squared_residuals

In [ ]:
# Determine the optimal LOWESS smoothing parameter, f, that minimizes the sum of the squared residuals
def smoothing_parameter(df, variable_of_interest, smoothing_factor_default):
    if smoothing_factor_default == True:
        optimal_fraction = 1
    if smoothing_factor_default == False:
        mses = pd.DataFrame()
        fold = 0
        
        # Perform a 5-Fold Cross-Validation
        kf = KFold(n_splits=5, shuffle=True, random_state=0) 
        for train_index, val_index in kf.split(df):
            # Separate each array into respective variables
            df_fold_train = df.iloc[train_index]
            df_fold_val = df.iloc[val_index]

            MSE_array = []
            interval = 0.05
            fractions_considered = np.arange(0.2,1+interval,interval)
            for fraction in fractions_considered:
                # Fit LOWESS model
                lowess = sm.nonparametric.lowess(df_fold_train['normalized_existing_installation'], df_fold_train[variable_of_interest], frac=fraction)
                lowess_x = list(zip(*lowess))[0]
                lowess_y = list(zip(*lowess))[1]
                f = interp1d(lowess_x, lowess_y, bounds_error=False) 
                y_pred = f(df_fold_val[variable_of_interest])
            
                # Calculate the mean squared error
                #   If the validation set has data points outside the range of the training set, 
                #   an error would occur in the calculation of the MSE. Therefore, validation points outside
                #   the training range are removed for this calculation
                if np.isnan(y_pred).any()==True:
                    y_comparison = pd.DataFrame({'y_pred': np.array(y_pred), 'y_fold_val': np.array(df_fold_val['normalized_existing_installation'])}, columns=['y_pred', 'y_fold_val'])
                    y_comparison = y_comparison.dropna()
                    MSE = mean_squared_error(y_comparison.y_fold_val, y_comparison.y_pred)
                else:
                    MSE = mean_squared_error(df_fold_val['normalized_existing_installation'], y_pred)
                MSE_array.append(MSE)

            mses[fold] = MSE_array
            fold = fold+1

        # Average the MSE across folds
        mses['mses_ave'] = mses.mean(axis=1)

        # Determine the minimum average MSE and with which fraction does it occur
        MSE_ave_min = min(mses.mses_ave)
        f_MSE_ave_min_index = mses['mses_ave'].idxmin()
        optimal_fraction = fractions_considered[f_MSE_ave_min_index]
        
    return optimal_fraction

In [ ]:
# Perform LOWESS on original race/ethnicity subset data
def lowess_original_data(df, variable_of_interest, smoothing_factor_default):
    optimal_fraction = smoothing_parameter(df, variable_of_interest, smoothing_factor_default)
    lowess = sm.nonparametric.lowess(df['normalized_existing_installation'], df[variable_of_interest], frac=optimal_fraction)
    lowess_x = list(zip(*lowess))[0]
    lowess_y = list(zip(*lowess))[1]
    f = interp1d(lowess_x, lowess_y, bounds_error=False)
    lowess_x_min = int(min(lowess_x)) 
    lowess_x_max = int(max(lowess_x))
    if variable_of_interest == 'median_income':
        xnew = list(range(lowess_x_min,lowess_x_max,50))
    else:
        xnew = list(range(lowess_x_min,lowess_x_max,1))
    y_original_data = f(xnew)
    return (xnew, y_original_data, optimal_fraction)

In [ ]:
# Perform LOWESS on a bootstrap replica of the race/ethnicity subset data
def lowess_bootstrap(df_sampled, variable_of_interest, xnew, optimal_fraction):
    lowess_sample = sm.nonparametric.lowess(df_sampled['normalized_existing_installation'], df_sampled[variable_of_interest], frac=optimal_fraction)
    lowess_x_sample = list(zip(*lowess_sample))[0]
    lowess_y_sample = list(zip(*lowess_sample))[1]
    f_sample = interp1d(lowess_x_sample, lowess_y_sample, bounds_error=False)
    y_sample = f_sample(xnew)
    return y_sample

In [ ]:
def lowess_confidence_interval(df, variable_of_interest, CI, smoothing_factor_default):
    # Perform LOWESS on the original race/ethnicity subset data
    (xnew, y_original_data, optimal_fraction) = lowess_original_data(df, variable_of_interest, smoothing_factor_default)
    Y = pd.DataFrame(y_original_data)

    # Perform LOWESS on 1000 bootstrap replicas of the original race/ethnicity subset data
    for i in range(1,1001):
        rows = np.random.choice(df.index.values, len(df))
        df_sampled = df.loc[rows]
        y_sample = lowess_bootstrap(df_sampled, variable_of_interest, xnew, optimal_fraction)
        Y.loc[:,i] = y_sample

    # Calculate the confidence interval band
    CI_high = Y.apply(lambda x: np.nanpercentile(x, (100 - (100-CI)/2)), axis=1)
    CI_low  = Y.apply(lambda x: np.nanpercentile(x, ((100-CI)/2)), axis=1)
    
    Y['xnew'] = xnew
    Y['CI_high'] = CI_high
    Y['CI_low'] = CI_low
    
    return (Y)

## Asian-majority

In [ ]:
Y_asian = lowess_confidence_interval(df_asian, variable_of_interest, CI, smoothing_factor_default)

In [ ]:
Y_asian.to_csv('Lowess_Asian_Results.csv')

## Black-majority

In [ ]:
Y_black = lowess_confidence_interval(df_black, variable_of_interest, CI, smoothing_factor_default)

In [ ]:
Y_black.to_csv('Lowess_Black_Results.csv')

## Hispanic-majority

In [ ]:
Y_hisp = lowess_confidence_interval(df_hisp, variable_of_interest, CI, smoothing_factor_default)

In [ ]:
Y_hisp.to_csv('Lowess_Hispanic_Results.csv')

## White-majority

In [ ]:
Y_white = lowess_confidence_interval(df_white, variable_of_interest, CI, smoothing_factor_default)

In [ ]:
Y_white.to_csv('Lowess_White_Results.csv')

## No-majority

In [ ]:
Y_no_majority = lowess_confidence_interval(df_no_majority, variable_of_interest, CI, smoothing_factor_default)

In [ ]:
Y_no_majority.to_csv('Lowess_No_Majority_Results.csv')

# CREATE FIGURE

## Subplot A

In [ ]:
# This subplot contains a histogram showing the number of census tracts by race/ethnicity as a function of the 
# variable of interest. Based on the user input, the variable of interest is either the median household income, 
# the percentage of households occupied by renters, or the surplus household income.

fig, axHistx = plt.subplots(figsize=(20, 5))

x_asian       = np.array(df_asian[variable_of_interest])
x_black       = np.array(df_black[variable_of_interest])
x_hisp        = np.array(df_hisp[variable_of_interest])
x_white       = np.array(df_white[variable_of_interest])
x_no_majority = np.array(df_no_majority[variable_of_interest])

if variable_of_interest == 'median_income':
    binwidth = 5000
    bins = np.arange(20000, 250000+binwidth, binwidth)
    axHistx.set_xlim([20000,260000])
    axHistx.set_xscale('log')
    fmt = '${x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    axHistx.xaxis.set_major_formatter(tick) 
    xticks = [25000,50000,75000,100000,150000,200000,250000]
    plt.xticks(xticks, fontsize = 30)
    plt.xticks(rotation = 45)
elif variable_of_interest == 'rental_ratio':
    binwidth = 5
    bins = np.arange(0, 100+binwidth, binwidth)
    axHistx.set_xlim([0,100])
elif variable_of_interest == 'surplus_income':
    binwidth = 5000
    bins = np.arange(np.floor(min(data['surplus_income'])), np.ceil(max(data['surplus_income']))+binwidth, binwidth)
    axHistx.set_xlim([np.floor(min(data['surplus_income'])),np.ceil(max(data['surplus_income']))+binwidth])
    fmt = '${x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    axHistx.xaxis.set_major_formatter(tick) 
    xticks = [-100000,-50000,0,50000,100000,150000,200000]
    plt.xticks(xticks, fontsize = 30)
    plt.xticks(rotation = 45)
else:
    print('Error: Variable of interest is inappropriately assigned')

axHistx.hist([x_white, x_hisp, x_black, x_asian, x_no_majority], bins =bins, stacked=True, color = ['blue', 'r', 'black', 'm','green'], alpha = 0.3)

axHistx.tick_params(axis='x', labelsize=25)
axHistx.tick_params(axis='y', labelsize=25)

axHistx.set_ylabel('Tracts', fontsize = 30, labelpad =15)

fig.tight_layout()

plt.show()

## Subplot B

In [ ]:
# This subplot shows the rooftop PV installations relative to the available rooftop PV potential normalized by state
# and grouped by the majority race/ethnicity as a function of the variable of interest. Based on the user input, the 
# variable of interest is either the median household income, the percentage of households occupied by renters, or the
# surplus household income.

fig, axLOWESS = plt.subplots(figsize=(20, 10))

if variable_of_interest == 'median_income':
    axLOWESS.set_xlabel('Median Household Income, 2013 USD', fontsize = 30, labelpad =15)
    axLOWESS.set_xlim([20000,260000])
    xticks = [25000,50000,75000,100000,150000,200000,250000]
    axLOWESS.set_xscale('log')
    fmt = '${x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    axLOWESS.xaxis.set_major_formatter(tick)
    plt.xticks(xticks, fontsize = 30)
    plt.xticks(rotation = 45)
elif variable_of_interest == 'rental_ratio':
    axLOWESS.set_xlabel('Percentage of households occupied by renters', fontsize = 30, labelpad =15)
    axLOWESS.set_xlim([0,100])
    xticks = [0,20,40,60,80,100]
    plt.xticks(xticks, fontsize = 30)
elif variable_of_interest == 'surplus_income':
    binwidth = 5000
    bins = np.arange(np.floor(min(data['surplus_income'])), np.ceil(max(data['surplus_income']))+binwidth, binwidth)
    axHistx.set_xlim([np.floor(min(data['surplus_income'])),np.ceil(max(data['surplus_income']))+binwidth])
    fmt = '${x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    axHistx.xaxis.set_major_formatter(tick) 
    xticks = [-100000,-50000,0,50000,100000,150000,200000]
    plt.xticks(xticks, fontsize = 30)
    plt.xticks(rotation = 45)
else:
    print('Error: Variable of interest is inappropriately assigned')
    
#No-majority 
axLOWESS.plot(Y_no_majority.xnew, Y_no_majority[0], '-', color = 'green', linewidth=2.0)
axLOWESS.fill_between(Y_no_majority.xnew, Y_no_majority.CI_low, Y_no_majority.CI_high, facecolor='green', alpha=0.3)

#Asian
axLOWESS.plot(Y_asian.xnew, Y_asian[0], '-', color = 'm', linewidth=2.0)
axLOWESS.fill_between(Y_asian.xnew, Y_asian.CI_low, Y_asian.CI_high, facecolor='m', alpha=0.3)

#Black
axLOWESS.plot(Y_black.xnew, Y_black[0], '-', color = 'black', linewidth=2.0)
axLOWESS.fill_between(Y_black.xnew, Y_black.CI_low, Y_black.CI_high, facecolor='black', alpha=0.3)

#Hispanic
axLOWESS.plot(Y_hisp.xnew, Y_hisp[0], '-', color = 'red', linewidth=2.0)
axLOWESS.fill_between(Y_hisp.xnew, Y_hisp.CI_low, Y_hisp.CI_high, facecolor='red', alpha=0.3)

#White
axLOWESS.plot(Y_white.xnew, Y_white[0], '-', color = 'blue', linewidth=2.0)
axLOWESS.fill_between(Y_white.xnew, Y_white.CI_low, Y_white.CI_high, facecolor='blue', alpha=0.3)

axLOWESS.set_ylabel('State Normalized \nSolar Deployment', fontsize = 30, labelpad =15)
axLOWESS.tick_params(axis='x', labelsize=25)
axLOWESS.tick_params(axis='y', labelsize=25)

upper_limit_y_axis = np.ceil(max([max(Y_no_majority.CI_high), max(Y_asian.CI_high), max(Y_black.CI_high), max(Y_hisp.CI_high), max(Y_white.CI_high)])) 
axLOWESS.set_ylim([0,upper_limit_y_axis])

# Create legend
asian_patch = mpatches.Patch(color='m', alpha = 0.3, label='Asian')
black_patch = mpatches.Patch(color='black', alpha = 0.3, label='Black')
hisp_patch = mpatches.Patch(color='r', alpha = 0.3, label='Hispanic')
white_patch = mpatches.Patch(color='blue', alpha = 0.3, label='White')
no_majority_patch = mpatches.Patch(color='green', alpha = 0.3, label='No-Majority')
plt.legend(handles=[no_majority_patch, asian_patch, black_patch, hisp_patch, white_patch],
               fontsize=25,loc='upper left', shadow=False, labelspacing =0.7)

fig.tight_layout()

plt.show()

## Subplot C

In [ ]:
# This subplot shows the state-normalized rooftop PV installations  normalized relative to the rooftop adoption of 
# no majority census tracts and grouped by the majority race/ethnicity as a function of the variable of interest. 
# Based on the user input, the variable of interest is either the median household income, the percentage of households
# occupied by renters, or the surplus household income.

fig, ax = plt.subplots(figsize=(20, 10))

# Installations relative to no-majority census tracts
lower_bound = int(min(Y_no_majority.xnew))
upper_bound = int(max(Y_no_majority.xnew))

if variable_of_interest == 'median_income':
    ax.set_xlabel('Median Household Income, 2013 USD', fontsize = 30, labelpad =15)
    ax.set_xlim([20000,260000])
    xticks = [25000,50000,75000,100000,150000,200000,250000]
    ax.set_xscale('log')
    fmt = '${x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    ax.xaxis.set_major_formatter(tick)
    plt.xticks(xticks, fontsize = 30)
    plt.xticks(rotation = 45)
    xvals = np.linspace(lower_bound, upper_bound, int((upper_bound-lower_bound)/50))
elif variable_of_interest == 'rental_ratio':
    axLOWESS.set_xlabel('Percentage of households occupied by renters', fontsize = 30, labelpad =15)
    axLOWESS.set_xlim([0,100])
    xticks = [0,20,40,60,80,100]
    plt.xticks(xticks, fontsize = 30)
    xvals = np.linspace(lower_bound, upper_bound, 100)
elif variable_of_interest == 'surplus_income':
    binwidth = 5000
    bins = np.arange(np.floor(min(data['surplus_income'])), np.ceil(max(data['surplus_income']))+binwidth, binwidth)
    axHistx.set_xlim([np.floor(min(data['surplus_income'])),np.ceil(max(data['surplus_income']))+binwidth])
    fmt = '${x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    axHistx.xaxis.set_major_formatter(tick) 
    xticks = [-100000,-50000,0,50000,100000,150000,200000]
    plt.xticks(xticks, fontsize = 30)
    plt.xticks(rotation = 45)
    xvals = np.linspace(lower_bound, upper_bound, int((upper_bound-lower_bound)/50))
else:
    print('Error: Variable of interest is inappropriately assigned')


# No-majority
yinterp_no_majority = np.interp(xvals, Y_no_majority.xnew, Y_no_majority[0])
relative_group  = yinterp_no_majority
yinterp_no_majority_CI_low = np.interp(xvals, Y_no_majority.xnew, Y_no_majority.CI_low)
yinterp_no_majority_CI_high = np.interp(xvals, Y_no_majority.xnew, Y_no_majority.CI_high)
relative_advantage_no_majority = (yinterp_no_majority-relative_group)/relative_group*100
relative_advantage_no_majority_CI_low = (yinterp_no_majority_CI_low-relative_group)/relative_group*100
relative_advantage_no_majority_CI_high = (yinterp_no_majority_CI_high-relative_group)/relative_group*100
ax.plot(xvals,relative_advantage_no_majority, color = 'green', linewidth=2.0)
ax.fill_between(xvals, relative_advantage_no_majority_CI_low, relative_advantage_no_majority_CI_high, facecolor='green', alpha=0.25)

# Asian
xvals_new = xvals[np.logical_and(xvals>=min(Y_asian.xnew),xvals<max(Y_asian.xnew))]
yinterp_no_majority_new = np.interp(xvals_new, Y_no_majority.xnew, Y_no_majority[0])
relative_group_new  = yinterp_no_majority_new
yinterp_asian = np.interp(xvals_new, Y_asian.xnew, Y_asian[0])
yinterp_asian_CI_low = np.interp(xvals_new, Y_asian.xnew, Y_asian.CI_low)
yinterp_asian_CI_high = np.interp(xvals_new, Y_asian.xnew, Y_asian.CI_high)
relative_advantage_asian = (yinterp_asian-relative_group_new)/relative_group_new*100
relative_advantage_asian_CI_low = (yinterp_asian_CI_low-relative_group_new)/relative_group_new*100
relative_advantage_asian_CI_high = (yinterp_asian_CI_high-relative_group_new)/relative_group_new*100
ax.plot(xvals_new,relative_advantage_asian, color = 'm', linewidth=2.0)
ax.fill_between(xvals_new, relative_advantage_asian_CI_low, relative_advantage_asian_CI_high, facecolor='m', alpha=0.25)

# Black
xvals_new = xvals[np.logical_and(xvals>=min(Y_black.xnew),xvals<max(Y_black.xnew))]
yinterp_no_majority_new = np.interp(xvals_new, Y_no_majority.xnew, Y_no_majority[0])
relative_group_new  = yinterp_no_majority_new
yinterp_black = np.interp(xvals_new, Y_black.xnew, Y_black[0])
yinterp_black_CI_low = np.interp(xvals_new, Y_black.xnew, Y_black.CI_low)
yinterp_black_CI_high = np.interp(xvals_new, Y_black.xnew, Y_black.CI_high)
relative_advantage_black = (yinterp_black-relative_group_new)/relative_group_new*100
relative_advantage_black_CI_low = (yinterp_black_CI_low-relative_group_new)/relative_group_new*100
relative_advantage_black_CI_high = (yinterp_black_CI_high-relative_group_new)/relative_group_new*100
ax.plot(xvals_new,relative_advantage_black, color = 'black', linewidth=2.0)
ax.fill_between(xvals_new, relative_advantage_black_CI_low, relative_advantage_black_CI_high, facecolor='black', alpha=0.25)

# Hispanic
xvals_new = xvals[np.logical_and(xvals>=min(Y_hisp.xnew),xvals<max(Y_hisp.xnew))]
yinterp_no_majority_new = np.interp(xvals_new, Y_no_majority.xnew, Y_no_majority[0])
relative_group_new  = yinterp_no_majority_new
yinterp_hisp = np.interp(xvals_new, Y_hisp.xnew, Y_hisp[0])
yinterp_hisp_CI_low = np.interp(xvals_new, Y_hisp.xnew, Y_hisp.CI_low)
yinterp_hisp_CI_high = np.interp(xvals_new, Y_hisp.xnew, Y_hisp.CI_high)
relative_advantage_hisp = (yinterp_hisp-relative_group_new)/relative_group_new*100
relative_advantage_hisp_CI_low = (yinterp_hisp_CI_low-relative_group_new)/relative_group_new*100
relative_advantage_hisp_CI_high = (yinterp_hisp_CI_high-relative_group_new)/relative_group_new*100
ax.plot(xvals_new,relative_advantage_hisp, color = 'r', linewidth=2.0)
ax.fill_between(xvals_new, relative_advantage_hisp_CI_low, relative_advantage_hisp_CI_high, facecolor='r', alpha=0.25)

# White
xvals_new = xvals[np.logical_and(xvals>=min(Y_white.xnew),xvals<max(Y_white.xnew))]
yinterp_no_majority_new = np.interp(xvals_new, Y_no_majority.xnew, Y_no_majority[0])
relative_group_new  = yinterp_no_majority_new
yinterp_white = np.interp(xvals_new, Y_white.xnew, Y_white[0])
yinterp_white_CI_low = np.interp(xvals_new, Y_white.xnew, Y_white.CI_low)
yinterp_white_CI_high = np.interp(xvals_new, Y_white.xnew, Y_white.CI_high)
relative_advantage_white = (yinterp_white-relative_group_new)/relative_group_new*100
relative_advantage_white_CI_low = (yinterp_white_CI_low-relative_group_new)/relative_group_new*100
relative_advantage_white_CI_high = (yinterp_white_CI_high-relative_group_new)/relative_group_new*100
ax.plot(xvals_new,relative_advantage_white, color = 'blue', linewidth=2.0)
ax.fill_between(xvals_new, relative_advantage_white_CI_low, relative_advantage_white_CI_high, facecolor='blue', alpha=0.25)

ax.set_ylabel('Solar Deployment Relative\nto No-Majority Census Tracts', fontsize = 30, labelpad =15)
ax.tick_params(axis='y', labelsize=25)

fig.tight_layout()

plt.show()

## Calculate Relative Advantage

In [ ]:
np.nanmean(relative_advantage_black)

In [ ]:
np.nanmean(relative_advantage_hisp)

In [ ]:
np.nanmean(relative_advantage_white)

In [ ]:
np.nanmean(relative_advantage_asian) 